In [14]:
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
import torch

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained('bert-base-uncased')

In [24]:
from tkinter import *
from PIL import ImageTk, Image
import torch

import nbimporter
#import NewModel as NM

master = Tk()

# Read the Image
image = Image.open("yelp.png")
 
# Resize the image using resize() method
resize_image = image.resize((100, 100))
 
img = ImageTk.PhotoImage(resize_image)
 
# create label and add resize image
label1 = Label(image=img)
label1.image = img
label1.grid(row=0)

Label(master, text='Enter Review: ').grid(row=1)
e1 = Entry(master)
e1.grid(row=2, column=0)

def buttonClick():
    
    #model = NM.model()
    
    review = e1.get()
    
    # Set up the layers and activation functions of our model
    model = torch.nn.Sequential(   
        torch.nn.Linear(768,800),
        torch.nn.SiLU(),
        torch.nn.Linear(800,64),
        torch.nn.SiLU(),
        torch.nn.Linear(64,64),
        torch.nn.SiLU(),
        torch.nn.Linear(64,64),
        torch.nn.SiLU(),
        torch.nn.Linear(64,64),
        torch.nn.SiLU(),
        torch.nn.Linear(64,64),
        torch.nn.SiLU(),
        torch.nn.Linear(64,16),
        torch.nn.SiLU(),
        torch.nn.Linear(16,1)
    )
    # Load the saved model
    model = torch.load("model.pth")

    # Set the model to evaluation mode
    #model.eval()
        
    input_sentence = review
    encoded_input = tokenizer.encode(input_sentence, add_special_tokens=False, return_tensors='pt')
    
   # model.eval(encoded_input)

    
    # Create a new input tensor
    x_new = torch.tensor([[0.5, 0.5]])

    # Use the model to make a prediction
    with torch.no_grad():
        y_pred = model(x_new)
        
        if y_pred < 0.5:
            y_pred = 0
        else:
            y_pred = 1

    print(y_pred)
        

button = Button(master, text="Enter", command=buttonClick)
button.grid(row=3, column=0)
        
master.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/kaijavanzante/opt/anaconda3/envs/py4sci/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/dg/jkq_f9b51f38pf3ddscgfrtm0000gn/T/ipykernel_63642/1477564838.py", line 50, in buttonClick
    y_pred = model(x_new)
             ^^^^^^^^^^^^
TypeError: 'collections.OrderedDict' object is not callable
